In [1]:
import sys
sys.path.append("../src")

import plotly.graph_objs as go
import plotly.offline as py

import utilities as u
import constants as c

In [2]:
import os
os.listdir("../src/app/pages")

['app_comparison.py',
 'app_evolution.py',
 'app_heatmaps.py',
 'app_pies.py',
 'app_violins.py',
 '__init__.py',
 '__pycache__']

In [4]:
exec("from utilities import ulog")

In [6]:
exec("log = ulog.set_logger()")

In [7]:
log.info("patata")

INFO     [2018-04-24 18:40:15]                      | patata


In [3]:
from app.pages import app_comparison

In [4]:
app_comparison.LINK

'/comparison'

In [2]:
from app import pages

In [3]:
import os
from flask import send_from_directory
from dash import Dash

import utilities as u
import constants as c
from app import layout


APP = Dash()
APP.config.supress_callback_exceptions = True
APP.css.config.serve_locally = True

DFG = u.uos.get_df(c.os.FILE_DATA_SAMPLE)
CATEGORIES = DFG[c.cols.CATEGORY].unique().tolist()

In [4]:
pages.get_pages(APP, DFG, CATEGORIES)

{'/comparison': ([Div(children=Graph(id='plot_comp_i', figure={'data': [{'type': 'scatter', 'x': Int64Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4], dtype='int64', name='Month_date'), 'y': array([1150, 1250, 1200, 1200, 1250, 1100, 1200, 1480, 1300, 1305, 1350,
          1540], dtype=int64), 'line': {'width': 1.0}, 'marker': {'color': '#66BB6A', 'size': 3}, 'name': 2017}, {'type': 'scatter', 'x': Int64Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4], dtype='int64', name='Month_date'), 'y': array([1380, 1300, 1350, 1330], dtype=int64), 'line': {'width': 1.5}, 'marker': {'color': '#1B5E20', 'size': 4}, 'name': 2018}], 'layout': {'title': 'Time comparison (Incomes)'}}, config={'displaylogo': False, 'modeBarButtonsToRemove': ['sendDataToCloud', 'select2d', 'lasso2d', 'resetScale2d']}), className='row', style={'text-align': 'center', 'padding-bottom': '15px', 'border-bottom': '2px solid #EEEEEE'}),
   Div(children=Graph(id='plot_comp_e', figure={'data': [{'type': 'scatt